<h1, align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;数据科学引论 - Python之道 </h1>

<h1, align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;第9课 机器学习</h1>
# <center>回归分析</center>


预测与分类类似，但是模型适合连续的/数值的/有序的值预测。

#### 关于本笔记
在这个笔记本中，我们下载了一个数据集，它与汽车的燃料消耗和二氧化碳排放有关。然后，我们将数据分解为训练和测试集，使用训练集创建模型，使用测试集评估模型，然后使用测试集验证模型。最后使用模型进行预测。


### 加载所需库

statsmomodels是一个Python模块，它允许用户查看数据、预测统计模型和执行测试。


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

### 下载数据

In [ ]:
#linux !wget -O /resources/FuelConsumption.csv https://ibm.box.com/shared/static/ez95yurarnp0q31l9jl1ma51mh6qtxj2.csv


## 了解数据

### `FuelConsumptionCo2.csv`:
我们下载了一个燃料消耗的数据集，包括了特定车型的燃油消耗率，以及在加拿大零售销售的新轻型车辆的二氧化碳排放量。 [数据源](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

- **MODELYEAR** 年份 e.g. 2014
- **MAKE**  制造商 e.g. Acura
- **MODEL** 车型 e.g. ILX
- **VEHICLE CLASS** 车类 e.g. SUV
- **ENGINE SIZE** 发动机大小 e.g. 4.7
- **CYLINDERS** 汽缸 e.g 6
- **TRANSMISSION** 变速器 e.g. A6
- **FUEL CONSUMPTION in CITY(L/100 km)** 城市道路百公里油耗 e.g. 9.9
- **FUEL CONSUMPTION in HWY (L/100 km)** 告诉公路百公里油耗 e.g. 8.9
- **FUEL CONSUMPTION COMB (L/100 km)** 综合路况百公里油耗 e.g. 9.2
- **CO2 EMISSIONS (g/km)**二氧化碳排放 e.g. 182  


## 读取数据

In [ ]:
df = pd.read_csv("resources/data/FuelConsumptionCo2.csv") 
# df = pd.read_csv("https://ibm.box.com/shared/static/ez95yurarnp0q31l9jl1ma51mh6qtxj2.csv")

# take a look at the dataset
df.head()

### 数据探索

In [ ]:
# summarize the data
df.describe()

In [ ]:
cdf=df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB','CO2EMISSIONS']]
cdf.head()

In [ ]:
viz=cdf[['CYLINDERS','ENGINESIZE','CO2EMISSIONS','FUELCONSUMPTION_COMB']]
viz.hist()
plt.show()

In [ ]:
plt.scatter(cdf.FUELCONSUMPTION_COMB, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("FUELCONSUMPTION_COMB")
plt.ylabel("Emission")
plt.show()

In [ ]:
plt.scatter(cdf.ENGINESIZE, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

In [ ]:
plt.scatter(cdf.CYLINDERS, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("Cylinders")
plt.ylabel("Emission")
plt.show()

#### 创建训练和测试数据集

In [ ]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

### 简单回归模型
### 关于线性回归
线性回归是一个线性模型，其系数为B = (B1, ..., Bn)，目的是最小化拟合曲线在每个x上的对应的预测y值与实际y之间的方差的和。

#### 训练数据的分布

In [ ]:
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

#### 建模
使用sklearn对数据建模

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
train_x=np.asanyarray(train[['ENGINESIZE']])
train_y=np.asanyarray(train[['CO2EMISSIONS']])
regr.fit (train_x, train_y)
# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ', regr.intercept_)

#### 绘制输出

In [ ]:
train_y_=regr.predict(train_x)
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue')
plt.plot(train_x, train_y_ , color='black',linewidth=3)

#### 评估
使用测试数据评估模型

In [ ]:
test_x=np.asanyarray(test[['ENGINESIZE']])
test_y=np.asanyarray(test[['CO2EMISSIONS']])
test_y_=regr.predict(test_x)


print("Residual mean of squares: %.2f"
      % np.mean((test_y_ - test_y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(test_x, test_y))

# Plot outputs
plt.scatter(test_x, test_y,  color='blue')
plt.plot(test_x, test_y_, color='black', linewidth=3)
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

__解释方差回归分数（explained variance regression score）:__  
如果 $\hat{y}$ 是预测的目标输出, y 是对应的正确的目标输出,Var是方差(标准差的平方), 那么解释方差回归分数如下：

$\texttt{explainedVariance}(y, \hat{y}) = 1 - \frac{Var\{ y - \hat{y}\}}{Var\{y\}}$  
最好的分数是1.0，越低表示拟合效果越差。

### 非线性回归

多项式回归是线性回归的一个特例。主要取决于你如何选择你的特征变量。看看有两个变量的多元回归:x1和x2。线性回归看起来是这样的:y=a1 \* x1 + a2 \* x2。

假设现在你想执行一个多项式回归（让我们创建一个2阶多项式），我们将添加一些额外的特征：x1\*x2, x1^2 和 x2^2，那么你的线性回归应该是：
y = a1 \* x1 + a2 \* x2 + a3 \* x1 \*x2 + a4 \* x1^2 + a5 \* x2^2

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

test_x=np.asanyarray(test[['ENGINESIZE']])
test_y=np.asanyarray(test[['CO2EMISSIONS']])


poly = PolynomialFeatures(degree=2)
train_x_poly = poly.fit_transform(train_x)

In [ ]:
clf = linear_model.LinearRegression()
train_y_ = clf.fit(train_x_poly, train_y)

### 测试

In [ ]:
test_x_poly = poly.fit_transform(test_x)
test_y_ = clf.predict(test_x_poly)

In [ ]:
plt.scatter(test_x, test_y,  color='blue')
plt.scatter(test_x, test_y_, linewidth=2)

### 多元回归模型


In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
x=np.asanyarray(train[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])
y=np.asanyarray(train[['CO2EMISSIONS']])
regr.fit (x, y)
# The coefficients
print('Coefficients: ', regr.coef_)
print ('Intercept: ', regr.intercept_)

In [ ]:
y_=regr.predict(test[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])
x=np.asanyarray(test[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])
y=np.asanyarray(test[['CO2EMISSIONS']])
print("Residual sum of squares: %.2f"
      % np.mean((y_ - y) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))